# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [118]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [36]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [37]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [38]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [39]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [40]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [41]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
# YOUR CODE HERE!
k_values = range(1, 11)

In [42]:
# Create an empy list to store the inertia values
# YOUR CODE HERE!
inertia_values = []

In [43]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
# YOUR CODE HERE!
for k in k_values:
    # Create a KMeans model with k clusters
    kmeans = KMeans(n_clusters=k, random_state=0)
    
    # Fit the model to the scaled data
    kmeans.fit(df_market_data_scaled)
    
    # Append the model's inertia to the inertia list
    inertia_values.append(kmeans.inertia_)    

/home/kay/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kay/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kay/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kay/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_

In [45]:
# Create a dictionary with the data to plot the Elbow curve
# YOUR CODE HERE!
elbow_data = {
    "K": k_values,
    "Inertia": inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
# YOUR CODE HERE!
df_elbow = pd.DataFrame(elbow_data)

In [81]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# YOUR CODE HERE!
df_elbow.hvplot.line(x="K", y="Inertia", title="Elbow Curve", xticks=k)

:Curve   [K]   (Inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** # 4

---

## Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for k found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with the best value for k.

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data.

4. View the resulting array of cluster values.

5. Create a copy of the original data and add a new column with the predicted clusters.

6. Create a scatter plot using hvPlot by setting x="price_change_percentage_24h" and y="price_change_percentage_7d".

7. Color the graph points with the labels found using K-Means.

8. Add the crypto name in the hover_cols parameter to identify the cryptocurrency represented by each data point.

    

In [47]:
# Initialize the K-Means model using the best value for k
# YOUR CODE HERE!
best_k = 4
kmeans = KMeans(n_clusters=best_k, random_state=0)

In [73]:
# Fit the K-Means model using the scaled data
# YOUR CODE HERE!
kmeans.fit(df_market_data_scaled)

/home/kay/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=4, random_state=0)

In [74]:
# Predict the clusters to group the cryptocurrencies using the scaled data
# YOUR CODE HERE!
stock_clusters = kmeans.predict(df_market_data_scaled)

# View the resulting array of cluster values.
# YOUR CODE HERE!
stock_clusters

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0],
      dtype=int32)

In [75]:
# Create a copy of the DataFrame
# YOUR CODE HERE!
df_market_data_scaled_copy = df_market_data_scaled.copy()

In [76]:
# Add a new column to the DataFrame with the predicted clusters
# YOUR CODE HERE!
df_market_data_scaled_copy["StockCluster"] = stock_clusters

# Display sample data
df_market_data_scaled_copy.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,StockCluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,2
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284,2
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428,2
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351,2


In [121]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
# YOUR CODE HERE!
scatter_plot = df_market_data_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    c=stock_clusters,  
    hover_cols=["coin_id"],  
    title="Cryptocurrency Clusters"
)
scatter_plot

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,_color,coin_id)

---

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [83]:
# Create a PCA model instance and set `n_components=3`.
# YOUR CODE HERE!
pca = PCA(n_components=3)

In [85]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
# YOUR CODE HERE!
principal_components = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
# YOUR CODE HERE!
df_principal_components = pd.DataFrame(data=principal_components, columns=['PCA1', 'PCA2', 'PCA3'], index=df_market_data.index)
df_principal_components.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


In [89]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
# YOUR CODE HERE!
explained_variance = pca.explained_variance_ratio_
total_explained_variance = sum(explained_variance)
total_explained_variance

0.8950316570309841

#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.8950316570309841

In [93]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you

# Creating a DataFrame with the PCA data
# YOUR CODE HERE!
df_pca = pd.DataFrame(data=principal_components, columns=['PCA1', 'PCA2', 'PCA3'], index=df_market_data.index)

# Copy the crypto names from the original data
# YOUR CODE HERE!
df_pca["coin_id"] = df_market_data.index

# Set the coinid column as index
# YOUR CODE HERE!
df_pca = df_pca.set_index("coin_id")

# Display sample data
# YOUR CODE HERE!
df_pca.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [94]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
# YOUR CODE HERE!
k_values = list(range(1, 12))

In [95]:
# Create an empy list to store the inertia values
# YOUR CODE HERE!
inertia_values = []

In [96]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
# YOUR CODE HERE!
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(df_pca)
    inertia_values.append(kmeans.inertia_)

/home/kay/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kay/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kay/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kay/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_

In [109]:
# Create a dictionary with the data to plot the Elbow curve
# YOUR CODE HERE!
elbow_data = {
    "K Values": k_values,
    "Inertia": inertia_values
}
# Create a DataFrame with the data to plot the Elbow curve
# YOUR CODE HERE!
df_elbow_pca = pd.DataFrame(elbow_data)

In [110]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# YOUR CODE HERE!
df_elbow_pca = pd.DataFrame({"K Values": k_values, "Inertia": inertia_values})
elbow_plot = df_elbow.hvplot.line(x="K Values", y="Inertia", title="Elbow Curve", xticks=k_values)
elbow_plot

:Curve   [K Values]   (Inertia)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [99]:
# Initialize the K-Means model using the best value for k
# YOUR CODE HERE!
best_k = 4 

In [100]:
# Fit the K-Means model using the PCA data
# YOUR CODE HERE!
kmeans_pca = KMeans(n_clusters=best_k, random_state=0)

In [102]:
# Predict the clusters to group the cryptocurrencies using the PCA data
# YOUR CODE HERE!
kmeans_pca.fit(df_pca)
cluster_labels_pca = kmeans_pca.predict(df_pca)

# View the resulting array of cluster values.
# YOUR CODE HERE!
cluster_labels_pca

/home/kay/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0],
      dtype=int32)

In [105]:
# Create a copy of the DataFrame with the PCA data
# YOUR CODE HERE!
df_pca_copy = df_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
# YOUR CODE HERE!
df_pca_copy["Cluster Labels"] = cluster_labels_pca

# Display sample data
# YOUR CODE HERE!
df_pca_copy.head()

,PCA1,PCA2,PCA3,Cluster Labels
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [117]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
# YOUR CODE HERE!
scatter_plot_pca_copy = df_pca_copy.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    c="Cluster Labels",  # Color points by cluster labels
    hover_cols=["coin_id"],  # Display cryptocurrency name on hover
    title="Cryptocurrency Clusters with PCA Data"
)

scatter_plot_pca_copy

:Scatter   [PCA1]   (PCA2,Cluster Labels,coin_id)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [116]:
# Composite plot to contrast the Elbow curves
composite_elbow_curves = df_elbow.hvplot.line(x="K Values", y="Inertia", title="Elbow Curve (Original Data)", xticks=k_values) + df_elbow.hvplot.line(x="K Values", y="Inertia", title="Elbow Curve (PCA Data)", xticks=k_values)

# Display the composite plot
composite_elbow_curves


:Layout
   .Curve.I  :Curve   [K Values]   (Inertia)
   .Curve.II :Curve   [K Values]   (Inertia)

In [123]:
# Compoosite plot to contrast the clusters
# YOUR CODE HERE!
composite_clusters = df_market_data.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    c=stock_clusters,  # Color points by cluster labels (original data)
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters (Original Data)"
) + df_pca_copy.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    c="Cluster Labels",  # Color points by cluster labels (PCA data)
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters (PCA Data)"
)

composite_clusters

:Layout
   .Scatter.I  :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,_color,coin_id)
   .Scatter.II :Scatter   [PCA1]   (PCA2,Cluster Labels,coin_id)

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 
  ## Impact of Using Fewer Features for Clustering

When use fewer features, such as principal components, to cluster data using K-Means, there are several potential impacts to consider:

- **Dimensionality Reduction:** Principal Component Analysis (PCA) reduces the dimensionality of the data by transforming it into a lower-dimensional space, which can help manage high-dimensional data more efficiently.

- **Improved Computation:** Clustering in a reduced feature space can improve the computational efficiency of the K-Means algorithm, making it faster and requiring less memory.

- **Potential Loss of Information:** PCA reduces the data to a subset of principal components, which may result in a loss of information. Features that are less influential in explaining variance may be discarded, affecting the interpretability of clusters.

- **Different Clustering Patterns:** Clustering in the reduced feature space may lead to different cluster patterns compared to clustering in the original feature space. Some clusters may become more distinct, while others may merge or exhibit different characteristics.

- **Interpretability:** Interpreting clusters in the reduced feature space can be more challenging, as the original features are transformed into principal components, which may not have clear, human-interpretable meanings.

- **Impact on Best Value for k:** The best value for k (number of clusters) may differ when using PCA data compared to the original data. It's essential to reevaluate the optimal k value when working with reduced features.

In summary, using PCA to reduce the number of features for clustering can provide computational benefits and reveal different cluster patterns. However, it also involves a trade-off with potential loss of information and interpretability. The choice to use PCA or not depends on the specific goals of the analysis and the trade-offs you are willing to make.
